# Obtain face embeddings using VGGFace model

In [12]:
import torch
import pickle
import sys, os

import importlib

import numpy as np
import cv2
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [2]:
project_dir = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
if not project_dir in sys.path: sys.path.append(project_dir)

In [3]:
def load_vggface_model(arch, project_dir, weights_path):
    '''
    Load either a resnet50 or senet50 model
    with pre-trained VGGFace weights
    '''
    # load model
    if arch == 'resnet':
        resnet = importlib.import_module('src.VGGFace2-pytorch.models.resnet', project_dir)
        vggface = resnet.resnet50(num_classes=8631, include_top=True)
    elif arch == 'senet':
        senet = importlib.import_module('src.VGGFace2-pytorch.models.senet', project_dir)
        vggface = senet.senet50(num_classes=8631, include_top=True)
    else:
        raise Exception('Wrong architecture, should be resnet or senet.')
    # load weights into model
    with open(weights_path, 'rb') as f:
        weights = pickle.load(f, encoding='latin1')
    own_state = vggface.state_dict()
    for name, param in weights.items():
        if name in own_state:
            if name != 'fc':
                try:
                    own_state[name].copy_(torch.from_numpy(param))
                except Exception:
                    raise RuntimeError('While copying the parameter named {}, whose dimensions in the model are {} and whose '\
                                       'dimensions in the checkpoint are {}.'.format(name, own_state[name].size(), param.size()))
        else:
            raise KeyError('unexpected key "{}" in state_dict'.format(name))
    return vggface

In [6]:
weights_path = '../models/external/resnet50_ft_weight.pkl'

vggface = load_vggface_model('resnet', project_dir, weights_path).cuda()



vggface = nn.Sequential(*(list(vggface.children())[:-1]))
print(vggface)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1)

In [7]:
FILE = '../data/interim/embeddings/juul_day_mtcnn.hdf5'

In [8]:
from src.utils import load_hdf

In [10]:
image_paths = load_hdf(FILE, keys=['image_paths'], print_results=True)

Dataset image_paths loaded with 385407 lines


In [36]:
mean_bgr = np.array([91.4953, 103.8827, 131.0912])

class JuulFaces(Dataset):
    def __init__(self, image_paths):
        super().__init__()
        self.image_paths = image_paths
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        path = self.image_paths[idx].decode('utf-8')
        img = cv2.imread(path)
        img = cv2.resize(img, (224,224), cv2.INTER_AREA)
        img = img.astype(np.float32)
        img -= mean_bgr
        img = img.transpose(2, 0, 1)  # C x H x W
        img = torch.from_numpy(img).float()
        return img

In [37]:
juul_faces = JuulFaces(image_paths['image_paths'])

loader = DataLoader(juul_faces, batch_size=64, num_workers=4)

In [38]:
for x in loader: break

In [40]:
x.size(), x.mean()

(torch.Size([64, 3, 224, 224]), tensor(-48.1524))

In [28]:
image_paths['image_paths'][0]

b'/home/neuro/RG/juul/process/data/interim/juul_day_mtcnn/1562310001_255_0.jpg'